In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Parquet Reader").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 15:03:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 15:03:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Initialize the airline variable globally
airline = None

def read_parquet_and_print_top(parquet_path):
    global airline
    # Read Parquet files into a DataFrame
    airline = spark.read.parquet(parquet_path)

# Initialize Spark session and read Parquet file
parquet_path = "./airline.parquet"
read_parquet_and_print_top(parquet_path)

In [4]:
# get cols 
print(airline.columns)

['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Reporting_Airline', 'DOT_ID_Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime'

In [7]:
flights_month = {}

months = airline.select('Month').distinct().collect()
months = [row.Month for row in months]
# assign total and delayed for each month
for month in months:
    total = airline.filter(airline.Month == month).count()
    delayed = airline.filter(airline.Month == month).filter(airline.ArrDelay > 0).count()
    flights_month[month] = {'total': total, 'delayed': delayed}
print(flights_month)

{1: {'total': 16415899, 'delayed': 7126891}, 3: {'total': 17023433, 'delayed': 7240339}, 9: {'total': 15541154, 'delayed': 5742708}, 4: {'total': 15804418, 'delayed': 6536136}, 8: {'total': 16759468, 'delayed': 7345132}, 7: {'total': 16744290, 'delayed': 7546039}, 2: {'total': 15115366, 'delayed': 6535627}, 12: {'total': 16249165, 'delayed': 7784369}, 10: {'total': 16572685, 'delayed': 6822165}, 11: {'total': 15741814, 'delayed': 6411187}, 6: {'total': 16195190, 'delayed': 7554970}, 5: {'total': 16222754, 'delayed': 6756295}}


24/03/16 17:30:08 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 597281 ms exceeds timeout 120000 ms
24/03/16 17:30:08 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/16 17:30:09 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$